In [1]:
import anndata
import numpy as np
import sys
import torch
sys.path.append('../../../../')
import velovae as vv

In [2]:
dataset = "Hindbrain_pons"
adata = anndata.read_h5ad(f'data/{dataset}_pp.h5ad')
adata.obs['cluster']=adata.obs['celltype'].to_numpy()

In [ ]:
vv.preprocess(adata, n_gene=2000, min_shared_counts=20)

In [3]:
model_path_base = f'checkpoints/{dataset}'
figure_path_base = f'figures/{dataset}'
data_path = f'data/velovae/discrete/{dataset}'
gene_plot = ['Ptprz1','Enpp6','Rras2','Mal']

# Discrete VeloVAE

In [6]:
figure_path = f'{figure_path_base}/DVAE'
model_path = f'{model_path_base}/DVAE'

torch.manual_seed(2022)
np.random.seed(2022)
dvae = vv.DVAE(adata, 
               tmax=20, 
               dim_z=5, 
               device='cuda:0',
               init_method='random')

dvae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

dvae.save_model(model_path, 'encoder', 'decoder')
dvae.save_anndata(adata, 'dvae', data_path, file_name=f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Random Initialization.
Gaussian Prior.
Library scale (U): Max=4.91, Min=0.10, Mean=0.97
Library scale (S): Max=5.28, Min=0.24, Mean=1.00
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 35, test iteration: 68
*********       Stage 1: Early Stop Triggered at epoch 991.       *********
*********                      Stage  2                       *********


  0%|          | 0/6253 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.006
Average Set Size: 236
*********       Stage 2: Early Stop Triggered at epoch 1168.       *********
*********              Finished. Total Time =   0 h :  7 m : 56 s             *********
Final: Train ELBO = -941.872,           Test ELBO = -971.619
       Training MSE = 7.355, Test MSE = 8.335


# Discrete Full VB

In [7]:
figure_path = f'{figure_path_base}/FullVB'
model_path = f'{model_path_base}/FullVB'

torch.manual_seed(2022)
np.random.seed(2022)
full_vb = vv.DVAEFullVB(adata, 
                        tmax=20, 
                        dim_z=5, 
                        device='cuda:0',
                        init_method='random')

full_vb.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

full_vb.save_model(model_path, 'encoder', 'decoder')
full_vb.save_anndata(adata, 'dfullvb', data_path, file_name=f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Random Initialization.
Gaussian Prior.
Library scale (U): Max=4.91, Min=0.10, Mean=0.97
Library scale (S): Max=5.28, Min=0.24, Mean=1.00
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 35, test iteration: 68
*********       Stage 1: Early Stop Triggered at epoch 199.       *********
*********                      Stage  2                       *********


  0%|          | 0/6253 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.001
Average Set Size: 172
*********       Stage 2: Early Stop Triggered at epoch 240.       *********
*********              Finished. Total Time =   0 h :  1 m : 58 s             *********
Final: Train ELBO = -1009.649,           Test ELBO = -1030.642
       Training MSE = 16.506, Test MSE = 16.380


# Evaluation

In [ ]:
cluster_edges = [('COPs', 'NFOLs'), ('NFOLs', 'MFOLs')]
vv.post_analysis(adata,
                 'eval',
                 ['Discrete VeloVAE','Discrete FullVB'],
                 ['dvae','dfullvb'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 plot_type=['all'],
                 save_path=data_path,
                 cluster_edges=cluster_edges,
                 sparsity_correction=True)